# Dijkstra's Algorithm
In this section of the coding homework, we will revise the famous Dijkstra's algorithms for finding shortest paths on weighted graphs with non-negative edges.

First, let us make some imports that will be helpful as we go on.

In [2]:
from heapq import heappush, heappop
import networkx as nx
import random

ok="""
         _          _                  _           _        
        /\ \       / /\               / /\        / /\      
       /  \ \     / /  \             / /  \      / /  \     
      / /\ \ \   / / /\ \           / / /\ \__  / / /\ \__  
     / / /\ \_\ / / /\ \ \         / / /\ \___\/ / /\ \___\ 
    / / /_/ / // / /  \ \ \        \ \ \ \/___/\ \ \ \/___/ 
   / / /__\/ // / /___/ /\ \        \ \ \       \ \ \       
  / / /_____// / /_____/ /\ \   _    \ \ \  _    \ \ \      
 / / /      / /_________/\ \ \ /_/\__/ / / /_/\__/ / /      
/ / /      / / /_       __\ \_\\ \/___/ /  \ \/___/ /       
\/_/       \_\___\     /____/_/ \_____\/    \_____\/        
                                                            
"""

## Statement of the problem
We are given an undirected weighted graph $G = (V, E, d)$, we denote the set of vertices by $V$, the set of edges by $E$ and we use $d$ to denote the weight of each edge. Note that for any edge $e \in E$, we have $d(e) \geq 0$. **Dijkstra's algorithm does not work for graphs that contain edges of negative weight**. We are also given a specific vertex $v \in V$. The problem asks us to find the length of the shortest path from $v$ to any other vertex $w \in V$ and to determine the length of this path. Dijkstra's algorithm solves this problem in $O((|V| + |E|) \log(|V|))$ time.

## Using a priority queue in Python

To implement Dijkstra's algorithm, we need to use a priority queue to store edges in the order of increasing edge weights. Let us look into how to use a priority queue in Python.

In [3]:
priority_queue = [] # To initialize a priority queue, start just with an array
heappush(priority_queue, 5) # Use heappush to push a new element to the priority queue
priority_queue

[5]

In [4]:
heappop(priority_queue) # Use heappop to pop a top element from the priority queue

5

In [5]:
priority_queue # The priority queue is now empty

[]

In [6]:
heappush(priority_queue, 6)
heappush(priority_queue, -5)
heappush(priority_queue, 7)
heappop(priority_queue)

-5

In [7]:
heappop(priority_queue)

6

In [8]:
heappop(priority_queue)

7

In [9]:
priority_queue # Now the priority queue is empty.

[]

We pushed elements in the order 6, -5, 7, and as we were popping top elements from the priority queue, they were popped in the order -5, 6, 7. Python maintains a **min priority queue**, which means that the top element is the **smallest** element in the priority queue.

Note that if we push pairs into the priority queue, they will be sorted by the first key.

In [10]:
heappush(priority_queue, (-1, 1))
heappush(priority_queue, (1, -1))
heappop(priority_queue) # (-1, 1) gets popped before (1, -1), since comparison happens by the first element

(-1, 1)

## Coding Dijkstra's Algorithm
If you need a refresher on how the algorithm works, check out pp.120-121 from DPV: https://people.eecs.berkeley.edu/~vazirani/algorithms/chap4.pdf

Now, let us get to coding the algorithm. 

__Note: Python's heapq does not have a decrease key operation so you will need to find a work around when implementing Dijkstra's. See the below section on space complexity for a hint on implementation.__

In [51]:
def shortest_path(graph, v):
  """
  - GRAPH is an adjacency list representation of the undirected graph.
    g[v] consists of tuples (u, d) such that (v, u) is an edge of weight d.
  - V is the start vertex from which we need to find the shortest distances.
  Return:
  - DISTANCE, a dictionary d such that d[u] is the length of the shortest path
    from V to u. By definition, d[V] = 0.
  - PARENT, a dictionary p such that p[u] is the parent of u on the shortest path
    from V to u. In other words, if the shortest path from V to u is (V, x, y, z, u),
    then p[u] = z, p[z] = y, ..., p[x] = V. We define p[V] to be None.
  """
  distance_and_vertex_priority_queue = []
  distance = {}
  parent = {}

  # Do any initialization of dictionaries or priority queue here

  """YOUR CODE HERE"""
  for u in graph.keys():
    distance[u] = float('inf')
    parent[u] = None
  
  distance[v] = 0
  heappush(distance_and_vertex_priority_queue, (distance[v], v))
  
  while distance_and_vertex_priority_queue: # What is the condition for the loop to finish?
    """
    In Python's implementation of a priority queue, it is difficult to change
    the key of an element that is already in the priority queue. This means that
    we cannot change a vertex's distance in the priority queue when the minimal
    distance to a vertex changes. For this reason, the priority queue might actually
    contain several entries for the same vertex. As we are taking out these
    entries we will check if they are "up-to-date". If the distance taken out of
    the priority queue is larger than the known minimal distance to this vertex,
    it means that this entry is out-of-date, so we can safely skip it.
    """
    distance_to_vertex, vertex = heappop(distance_and_vertex_priority_queue)
    if distance_to_vertex > distance[vertex]:
        continue
    
    """YOUR CODE HERE"""
    for u, weight in graph[vertex]:
      if distance[u] > distance_to_vertex + weight:
        distance[u] = distance_to_vertex + weight
        parent[u] = vertex
        heappush(distance_and_vertex_priority_queue, (distance[u], u))

  return distance, parent

In [59]:
def get_path(w, parent):
    """
    Take in a vertex W and a dictionary PARENT, as described in the docstring
    for the shortest_path method, and return the path from V to W, where V
    is the vertex from which we ran Dijkstra's algorithm.
    """
    
    """YOUR CODE HERE"""
    path = [w]
    while parent[w] != None:
        path.append(parent[w])
        w = parent[w]

    return list(reversed(path))

def get_distance(w, distance):
    """
    Take in a vertex W and a dictionary DISTANCE, as described in the docstring
    for the shortest_path method, and return the length of the shortest path from V to W,
    where V is the vertex from which we ran Dijkstra's algorithm.
    """
    return distance[w]

Now, let us verify that your algorithm works. If it passes the test, you will get a "PASS" message. If it fails, an AssertionError will be raised.

In [61]:
def verify_using_random_graphs():
    N = 20
    n = 50
    for i in range(N):
        nx_graph = nx.gnp_random_graph(n, 0.2)
        while not nx.is_connected(nx_graph):
            nx_graph = nx.gnp_random_graph(n, 0.2)

        for (u, v, w) in nx_graph.edges(data=True):
            w['weight'] = random.randint(0, 10000)
        adj_list_graph = {}
        for v in nx_graph.nodes:
            for u in nx_graph.adj[v]:
                if v not in adj_list_graph:
                    adj_list_graph[v] = []
                weight = nx_graph[v][u]['weight']
                adj_list_graph[v].append((u, weight))
        
        start_vertex = random.sample(nx_graph.nodes, k=1)[0]
        distance, parent = shortest_path(adj_list_graph, start_vertex)
        for vertex in nx_graph.nodes:
            our_path = get_path(vertex, parent)
            nx_distance = nx.dijkstra_path_length(nx_graph, start_vertex, vertex, 'weight')
            our_distance = get_distance(vertex, distance)
            assert nx_distance == our_distance
            assert our_path[0] == start_vertex
            assert our_path[-1] == vertex
            total_distance_of_path = 0
            for i in range(1, len(our_path)):
                from_v = our_path[i - 1]
                to_v = our_path[i]
                assert (from_v, to_v) in nx_graph.edges()
                total_distance_of_path += nx_graph[from_v][to_v]['weight']
            assert total_distance_of_path == nx_distance

    print(ok)

verify_using_random_graphs()


         _          _                  _           _        
        /\ \       / /\               / /\        / /\      
       /  \ \     / /  \             / /  \      / /  \     
      / /\ \ \   / / /\ \           / / /\ \__  / / /\ \__  
     / / /\ \_\ / / /\ \ \         / / /\ \___\/ / /\ \___\ 
    / / /_/ / // / /  \ \ \        \ \ \ \/___/\ \ \ \/___/ 
   / / /__\/ // / /___/ /\ \        \ \ \       \ \ \       
  / / /_____// / /_____/ /\ \   _    \ \ \  _    \ \ \      
 / / /      / /_________/\ \ \ /_/\__/ / / /_/\__/ / /      
/ / /      / / /_       __\ \_\ \/___/ /  \ \/___/ /       
\/_/       \_\___\     /____/_/ \_____\/    \_____\/        
                                                            



# Greatest Roads
Here, we will implement an algorithm to solve the Greatest Roads question from HW 3. The following is the prompt for that question:
> Arguably, one of the best things to do in America is to take a great
American road trip. And in America there are some amazing roads to
drive on (think Pacific Coast Highway, Route 66 etc). An intrepid
traveler has chosen to set course across America in search of some
amazing driving. What is the length of the shortest path that hits at
least $k$ of these amazing roads?

> Assume that the roads in America can be expressed as a directed
weighted graph $G = (V,E, d)$, and that our traveler wishes to drive
across at least $k$ roads from the subset $R \subseteq E$ of ``amazing''
roads. Furthermore, assume that the traveler starts and ends at her
home $a \in V$. You may also assume that the traveler is fine with
repeating roads from $R$, i.e. the $k$ roads chosen from $R$ need not
be unique. 

> Design an efficient algorithm to solve this problem. 
Provide a 3-part solution with runtime in terms of $n = |V|$, $m =
|E|$, $k$. 

__Note: The following discussion of space complexity is not necessary for the completion of this problem; rather, it gives more context to how this problem can be implemented. There are two main implementations of the algorithm which we describe and analyze below. You are free to implement either or any other implementation you come up with regardless of the implementations described below.__

## Space Complexity

In this question, we will introduce the topic of space complexity. Broadly speaking, space complexity describes the maximum amount of memory an algorithm will use at any one point in time. Space complexity depends only on the space allocated when the algorithm is running; it does not depend on the size of the input. 

For example, binary search uses $O(1)$ space since at each point in time, only the left index, right index, and middle index are stored. Notice that the size of the array is not included in this analysis. Also notice this analysis only cares about the maximum space required at any one point in time, not how many times individual variables might be reassigned. I.E. space you allocated earlier in the algorithm can be reused for different purposes later on, and this does not count against the space complexity.

Now, let's analyze the space complexity of the above implementation of Dijkstra's algorithm. The above implementation allocates a `parent` and `distance` dictionary, each of which will hold at most $O(N)$ elements where N is the number of vertices in the graph. The implementation also allocates space for a heap which records `(distance, node)` pairs. Each node can be pushed onto the heap at most $O(\text{degree(node)})$ times.  Thus, the heap can grow to a size of $O(M)$ where $M$ is the number of edges in the graph. In each iteration of the loop, your implementation might also allocate a few temporary variables. Therefore, the algorithm will use $O(N + M)$ space to maintain all the dictionaries, heaps, and temporary variables. 

Note that there exist many different implementations of Dijkstra's algorithm, some of which take only $O(N)$ space. Although not directly relevant to Dijkstra's implementation in this notebook, keep this fact in mind for the following analysis of Greatest Roads. 

## Implementation 1 - Explicitly constructing the graph

The first implementation explicitly creates the graph described in the question then runs Dijkstra's on said graph. In summary, this implementation first allocates enough space for the adjacency list to create $k$ copies of both the edges and vertices. Then, it connects the copies of the graph via the greatest roads. Since this algorithm needs to allocate space for the entire graph, it uses $O(kE+kV))$ space where $V$ and $E$ are the number of vertices and edges in the original graph respectively. Then, we use a Dijkstra implementation which takes $O(N) = O(kV)$ space on the new graph we created. This algorithm will take $O(kV + kE)$ space overall. 

## Implementation 2 - Implicitly constructing the graph
In the second implementation, we won't explicitly construct the new graph with $O(kV)$ vertices and $O(kE)$ edges. Instead, we'll only explicitly create the $O(kV)$ vertices of $G'$, and lazily figure out the edges of $G'$ as Dijkstra's traverses the graph.

To achieve this, we will modify our implementation of Dijkstra's to handle the new graph as follows:
1. Rather than a normal distance list where `distance[v] = the distance to v`, we instead create a distance matrix by making a list of length k lists. Here, `distance_matrix[v][j] = the distance to node v in the j-th copy of the graph.` In order to reconstruct the path that Dijkstra's finds, we do a similar thing to create a parent matrix. 

2. Typically, we would push a tuple `(d,u)` into the heap, representing the distance to node u being d. Here, we will push in `(d,u,j)` representing the distance to node u in copy j being d. This tells the algorithm what copy of the graph it is currently traversing and which entry of distance_matrix it should set.

3. In order to find the neighbors of the current node, we must implicitly figure out all of the outgoing edges from this node. To do so, we check the original adjacency_list of G to find the outgoing edges for v. For each edge, if it is not a greatest road, then we must stay in the same copy of the graph (ie. if v is in copy j, its neighbor is also in copy j). If the road is a greatest road, then we go to the next copy (ie. if v is in copy j, it's neighbor is in copy j + 1).

4. If the current node is in copy k (the last one), then all edges point to other nodes in copy k exist regardless of whether or not they are greatest roads.

Although the construction of the graph is different, the algorithm still essentially performs the same steps: find the shortest path from a source node to other nodes, such that we only jump from one copy of the graph to the next if we are taking a greatest road. 

In this implementation we only use $O(kV)$ space to create the parents and distance matrix. As a result, this implementation will only use $O(kV)$ space, a significant improvement over the first implementation.

In [98]:
import numpy as np

def greatest_roads_solver(non_great_roads, greatest_roads, k, a, n):
    """
    Returns the shortest path which starts at node a and ends at node a which goes through k greatest roads
    
    args:
    non_great_roads: a list of tuples (u,v,d) containing all roads which are not greatest roads
    greatest_roads: a list of tuples containing all roads which are greatest roads
    k: an int representing the number of greatest roads the path must traverse
    a: the node representing home, which the path must start and end at
    n: the number of nodes in the graph
    """
    distance_and_vertex_priority_queue = []
    distance = {}
    parent = {}
    graph = {}

    for v, u, weight in non_great_roads + greatest_roads:
        if v not in graph.keys():
            graph[v] = []
        graph[v].append((u, weight))


    for u in range(n):
        distance[u] = [float('inf')] * (k+1)
        parent[u] = [None] * (k+1)
    
    distance[a][0] = 0
    heappush(distance_and_vertex_priority_queue, (distance[a][0], a, 0))

    while distance_and_vertex_priority_queue: 
        distance_to_vertex, vertex, copy = heappop(distance_and_vertex_priority_queue)
        if distance_to_vertex > distance[vertex][copy]:
            continue
        
        for u, weight in graph[vertex]:
            if (vertex, u, weight) in greatest_roads and copy < k:
                if distance[u][copy+1] > distance_to_vertex + weight:
                    distance[u][copy+1] = distance_to_vertex + weight
                    parent[u][copy+1] = (vertex, copy)
                    heappush(distance_and_vertex_priority_queue, (distance[u][copy+1], u, copy+1))
            else:
                if distance[u][copy] > distance_to_vertex + weight:
                    distance[u][copy] = distance_to_vertex + weight
                    parent[u][copy] = (vertex, copy)
                    heappush(distance_and_vertex_priority_queue, (distance[u][copy], u, copy))

    path = [a]
    node, parent_copy = a, k
    while parent[node][parent_copy] != None:
        path.append(parent[node][parent_copy][0])
        node, parent_copy = parent[node][parent_copy]


    return list(reversed(path))

# Verification

We verify your solution by first validating the paths indeed pass through k greatest roads then checking that they are the shortest path, based on the following graphs which we have precomputed.

In [66]:
greatest_roads_all = []
non_great_roads_all = []
path_all = []
greatest_roads_all.append( [(8, 2, {'weight': 75})] )
non_great_roads_all.append( [(0, 6, {'weight': 55}), (1, 4, {'weight': 80}), (2, 6, {'weight': 76}), (2, 8, {'weight': 66}), (2, 9, {'weight': 73}), (3, 5, {'weight': 7}), (4, 1, {'weight': 93}), (4, 8, {'weight': 65}), (5, 6, {'weight': 33}), (6, 7, {'weight': 44}), (7, 2, {'weight': 25}), (8, 3, {'weight': 46}), (8, 6, {'weight': 87}), (9, 0, {'weight': 70}), (9, 1, {'weight': 65}), (9, 2, {'weight': 57}), (9, 3, {'weight': 75})] )
path_all.append( [5, 6, 7, 2, 8, 2, 8, 2, 8, 2, 8, 2, 8, 2, 8, 3, 5] )
greatest_roads_all.append( [(7, 0, {'weight': 31}), (9, 0, {'weight': 90})] )
non_great_roads_all.append( [(0, 5, {'weight': 40}), (0, 9, {'weight': 68}), (1, 8, {'weight': 27}), (1, 9, {'weight': 50}), (2, 6, {'weight': 37}), (3, 7, {'weight': 22}), (4, 0, {'weight': 48}), (5, 3, {'weight': 57}), (5, 4, {'weight': 14}), (5, 9, {'weight': 61}), (6, 1, {'weight': 100}), (7, 2, {'weight': 80}), (7, 5, {'weight': 98}), (8, 6, {'weight': 25}), (8, 7, {'weight': 59}), (8, 9, {'weight': 72}), (9, 3, {'weight': 92}), (9, 5, {'weight': 20})] )
path_all.append( [5, 3, 7, 0, 5, 3, 7, 0, 5, 3, 7, 0, 5, 3, 7, 0, 5, 3, 7, 0, 5] )
greatest_roads_all.append( [(2, 7, {'weight': 52}), (0, 5, {'weight': 87})] )
non_great_roads_all.append( [(0, 7, {'weight': 95}), (0, 8, {'weight': 38}), (1, 5, {'weight': 62}), (1, 7, {'weight': 28}), (1, 8, {'weight': 24}), (2, 9, {'weight': 84}), (3, 1, {'weight': 10}), (3, 8, {'weight': 29}), (4, 0, {'weight': 68}), (4, 5, {'weight': 59}), (4, 6, {'weight': 25}), (4, 9, {'weight': 17}), (5, 3, {'weight': 69}), (5, 7, {'weight': 81}), (6, 2, {'weight': 58}), (6, 8, {'weight': 35}), (7, 6, {'weight': 43}), (8, 2, {'weight': 62}), (8, 4, {'weight': 24}), (8, 5, {'weight': 58}), (9, 1, {'weight': 46}), (9, 3, {'weight': 52}), (9, 5, {'weight': 57})] )
path_all.append( [5, 3, 8, 2, 7, 6, 2, 7, 6, 2, 7, 6, 2, 7, 6, 8, 4, 0, 5] )
greatest_roads_all.append( [(0, 9, {'weight': 48}), (8, 7, {'weight': 48})] )
non_great_roads_all.append( [(0, 1, {'weight': 29}), (0, 5, {'weight': 55}), (0, 8, {'weight': 72}), (1, 2, {'weight': 22}), (1, 3, {'weight': 67}), (1, 5, {'weight': 82}), (1, 6, {'weight': 12}), (1, 7, {'weight': 47}), (2, 1, {'weight': 47}), (3, 0, {'weight': 28}), (3, 9, {'weight': 42}), (4, 2, {'weight': 20}), (4, 8, {'weight': 88}), (5, 4, {'weight': 13}), (5, 6, {'weight': 75}), (5, 9, {'weight': 21}), (6, 7, {'weight': 75}), (6, 8, {'weight': 89}), (7, 4, {'weight': 93}), (7, 5, {'weight': 69}), (8, 4, {'weight': 52}), (9, 2, {'weight': 56}), (9, 3, {'weight': 74}), (9, 5, {'weight': 93})] )
path_all.append( [5, 9, 3, 0, 9, 3, 0, 9, 3, 0, 9, 3, 0, 9, 3, 0, 9, 5] )
greatest_roads_all.append( [(5, 8, {'weight': 72}), (6, 1, {'weight': 99})] )
non_great_roads_all.append( [(0, 2, {'weight': 59}), (0, 4, {'weight': 34}), (1, 3, {'weight': 55}), (1, 5, {'weight': 22}), (1, 7, {'weight': 54}), (1, 8, {'weight': 93}), (2, 9, {'weight': 84}), (3, 5, {'weight': 49}), (4, 3, {'weight': 91}), (5, 0, {'weight': 80}), (5, 4, {'weight': 40}), (7, 0, {'weight': 31}), (7, 1, {'weight': 89}), (7, 5, {'weight': 60}), (7, 8, {'weight': 15}), (8, 2, {'weight': 63}), (8, 5, {'weight': 71}), (8, 9, {'weight': 3}), (9, 2, {'weight': 22}), (9, 4, {'weight': 13}), (9, 6, {'weight': 56})] )
path_all.append( [5, 8, 9, 6, 1, 5, 8, 5, 8, 9, 6, 1, 5] )
greatest_roads_all.append( [(0, 4, {'weight': 6}), (5, 0, {'weight': 99})] )
non_great_roads_all.append( [(0, 2, {'weight': 63}), (0, 7, {'weight': 33}), (0, 9, {'weight': 92}), (1, 2, {'weight': 97}), (2, 1, {'weight': 89}), (2, 5, {'weight': 81}), (2, 8, {'weight': 95}), (3, 0, {'weight': 93}), (3, 5, {'weight': 92}), (4, 5, {'weight': 87}), (4, 7, {'weight': 10}), (6, 1, {'weight': 51}), (6, 3, {'weight': 19}), (6, 9, {'weight': 51}), (7, 0, {'weight': 86}), (7, 5, {'weight': 25}), (7, 6, {'weight': 9}), (8, 2, {'weight': 83}), (8, 3, {'weight': 51}), (8, 6, {'weight': 85}), (9, 8, {'weight': 40})] )
path_all.append( [5, 0, 4, 7, 5, 0, 4, 7, 0, 4, 7, 5] )
greatest_roads_all.append( [(6, 9, {'weight': 26}), (0, 3, {'weight': 79})] )
non_great_roads_all.append( [(0, 6, {'weight': 2}), (0, 8, {'weight': 100}), (1, 3, {'weight': 73}), (2, 0, {'weight': 91}), (2, 1, {'weight': 64}), (2, 5, {'weight': 91}), (2, 6, {'weight': 93}), (3, 1, {'weight': 49}), (3, 2, {'weight': 87}), (3, 7, {'weight': 28}), (4, 1, {'weight': 99}), (4, 8, {'weight': 48}), (5, 8, {'weight': 34}), (7, 0, {'weight': 70}), (7, 1, {'weight': 33}), (7, 4, {'weight': 71}), (8, 9, {'weight': 59}), (9, 1, {'weight': 76}), (9, 5, {'weight': 79})] )
path_all.append( [5, 8, 9, 1, 3, 7, 0, 3, 7, 0, 3, 7, 0, 3, 7, 0, 3, 7, 0, 6, 9, 5] )
greatest_roads_all.append( [(9, 7, {'weight': 92}), (5, 9, {'weight': 63})] )
non_great_roads_all.append( [(0, 5, {'weight': 71}), (1, 6, {'weight': 69}), (2, 0, {'weight': 50}), (2, 3, {'weight': 27}), (2, 6, {'weight': 40}), (2, 7, {'weight': 30}), (3, 2, {'weight': 64}), (3, 9, {'weight': 95}), (4, 1, {'weight': 20}), (4, 6, {'weight': 35}), (4, 9, {'weight': 41}), (5, 0, {'weight': 87}), (5, 1, {'weight': 48}), (5, 6, {'weight': 90}), (5, 8, {'weight': 44}), (6, 0, {'weight': 3}), (6, 9, {'weight': 58}), (7, 0, {'weight': 10}), (7, 4, {'weight': 40}), (8, 2, {'weight': 84}), (8, 5, {'weight': 21}), (9, 3, {'weight': 71}), (9, 6, {'weight': 28})] )
path_all.append( [5, 9, 7, 0, 5, 9, 7, 0, 5, 9, 6, 0, 5] )
greatest_roads_all.append( [(3, 0, {'weight': 18}), (8, 0, {'weight': 80})] )
non_great_roads_all.append( [(0, 1, {'weight': 2}), (1, 3, {'weight': 69}), (1, 5, {'weight': 59}), (1, 7, {'weight': 60}), (1, 8, {'weight': 45}), (2, 3, {'weight': 49}), (2, 4, {'weight': 51}), (2, 6, {'weight': 42}), (3, 2, {'weight': 70}), (3, 4, {'weight': 83}), (4, 0, {'weight': 48}), (4, 5, {'weight': 65}), (4, 6, {'weight': 66}), (4, 9, {'weight': 62}), (5, 0, {'weight': 13}), (5, 8, {'weight': 49}), (6, 5, {'weight': 81}), (6, 7, {'weight': 81}), (7, 1, {'weight': 7}), (7, 5, {'weight': 14}), (7, 6, {'weight': 24}), (8, 2, {'weight': 53}), (9, 2, {'weight': 17})] )
path_all.append( [5, 0, 1, 3, 0, 1, 3, 0, 1, 3, 0, 1, 3, 0, 1, 3, 0, 1, 5] )
greatest_roads_all.append( [(6, 7, {'weight': 7}), (9, 6, {'weight': 43})] )
non_great_roads_all.append( [(0, 1, {'weight': 97}), (0, 9, {'weight': 56}), (1, 2, {'weight': 2}), (1, 6, {'weight': 89}), (1, 8, {'weight': 45}), (2, 0, {'weight': 17}), (2, 5, {'weight': 68}), (2, 7, {'weight': 98}), (2, 9, {'weight': 93}), (3, 5, {'weight': 5}), (3, 8, {'weight': 51}), (4, 9, {'weight': 45}), (5, 3, {'weight': 33}), (6, 2, {'weight': 10}), (7, 5, {'weight': 23}), (8, 5, {'weight': 43}), (8, 6, {'weight': 37}), (9, 4, {'weight': 34})] )
path_all.append( [5, 3, 8, 6, 2, 0, 9, 6, 2, 0, 9, 6, 2, 0, 9, 6, 2, 0, 9, 6, 7, 5] )
greatest_roads_all.append( [(9, 8, {'weight': 44}), (9, 5, {'weight': 54})] )
non_great_roads_all.append( [(0, 1, {'weight': 23}), (0, 4, {'weight': 92}), (0, 7, {'weight': 27}), (1, 4, {'weight': 11}), (1, 6, {'weight': 74}), (1, 8, {'weight': 76}), (2, 0, {'weight': 98}), (2, 5, {'weight': 9}), (2, 6, {'weight': 46}), (3, 1, {'weight': 90}), (3, 2, {'weight': 15}), (3, 9, {'weight': 61}), (4, 6, {'weight': 92}), (4, 8, {'weight': 42}), (5, 3, {'weight': 54}), (6, 0, {'weight': 81}), (6, 4, {'weight': 56}), (7, 5, {'weight': 29}), (8, 0, {'weight': 23}), (8, 1, {'weight': 49}), (8, 7, {'weight': 35}), (8, 9, {'weight': 67}), (9, 4, {'weight': 33})] )
path_all.append( [5, 3, 9, 8, 9, 8, 9, 8, 9, 8, 9, 5] )
greatest_roads_all.append( [(4, 8, {'weight': 38}), (6, 0, {'weight': 82})] )
non_great_roads_all.append( [(0, 3, {'weight': 9}), (0, 5, {'weight': 63}), (1, 8, {'weight': 55}), (2, 1, {'weight': 88}), (2, 5, {'weight': 90}), (2, 9, {'weight': 24}), (3, 0, {'weight': 72}), (3, 8, {'weight': 78}), (4, 1, {'weight': 94}), (5, 8, {'weight': 95}), (6, 2, {'weight': 93}), (7, 1, {'weight': 83}), (7, 4, {'weight': 71}), (7, 5, {'weight': 64}), (7, 8, {'weight': 53}), (8, 5, {'weight': 69}), (8, 6, {'weight': 81}), (9, 1, {'weight': 67}), (9, 2, {'weight': 42}), (9, 3, {'weight': 23}), (9, 5, {'weight': 84}), (9, 7, {'weight': 38}), (9, 8, {'weight': 52})] )
path_all.append( [5, 8, 6, 0, 3, 8, 6, 0, 3, 8, 6, 0, 3, 8, 6, 0, 3, 8, 6, 0, 5] )
greatest_roads_all.append( [(2, 9, {'weight': 58}), (2, 6, {'weight': 78})] )
non_great_roads_all.append( [(0, 1, {'weight': 25}), (0, 3, {'weight': 16}), (0, 4, {'weight': 68}), (0, 5, {'weight': 4}), (0, 9, {'weight': 79}), (1, 4, {'weight': 8}), (1, 8, {'weight': 58}), (2, 3, {'weight': 8}), (2, 8, {'weight': 30}), (3, 0, {'weight': 50}), (3, 4, {'weight': 80}), (3, 9, {'weight': 97}), (4, 2, {'weight': 2}), (4, 7, {'weight': 63}), (5, 3, {'weight': 97}), (6, 0, {'weight': 93}), (6, 4, {'weight': 29}), (7, 0, {'weight': 52}), (7, 3, {'weight': 70}), (7, 5, {'weight': 86}), (7, 6, {'weight': 35}), (8, 2, {'weight': 39}), (8, 4, {'weight': 43}), (9, 0, {'weight': 24}), (9, 6, {'weight': 76})] )
path_all.append( [5, 3, 4, 2, 6, 4, 2, 6, 4, 2, 6, 4, 2, 6, 4, 2, 9, 0, 5] )
greatest_roads_all.append( [(4, 0, {'weight': 96})] )
non_great_roads_all.append( [(0, 2, {'weight': 77}), (0, 6, {'weight': 15}), (0, 8, {'weight': 98}), (1, 6, {'weight': 11}), (1, 7, {'weight': 42}), (2, 5, {'weight': 40}), (3, 1, {'weight': 44}), (3, 4, {'weight': 91}), (5, 9, {'weight': 64}), (6, 3, {'weight': 84}), (7, 1, {'weight': 14}), (7, 2, {'weight': 76}), (7, 8, {'weight': 9}), (7, 9, {'weight': 48}), (8, 5, {'weight': 90}), (8, 6, {'weight': 75}), (9, 1, {'weight': 1})] )
path_all.append( [5, 9, 1, 6, 3, 4, 0, 6, 3, 4, 0, 6, 3, 4, 0, 6, 3, 4, 0, 6, 3, 4, 0, 2, 5] )
greatest_roads_all.append( [(9, 6, {'weight': 61}), (5, 6, {'weight': 3})] )
non_great_roads_all.append( [(0, 1, {'weight': 86}), (0, 2, {'weight': 44}), (0, 3, {'weight': 40}), (0, 9, {'weight': 89}), (1, 3, {'weight': 2}), (2, 6, {'weight': 24}), (2, 7, {'weight': 69}), (3, 2, {'weight': 73}), (3, 4, {'weight': 21}), (4, 0, {'weight': 13}), (5, 1, {'weight': 11}), (6, 3, {'weight': 26}), (6, 4, {'weight': 56}), (6, 8, {'weight': 100}), (6, 9, {'weight': 53}), (7, 0, {'weight': 57}), (8, 3, {'weight': 16}), (8, 5, {'weight': 54}), (9, 0, {'weight': 55}), (9, 3, {'weight': 49}), (9, 4, {'weight': 63}), (9, 8, {'weight': 27})] )
path_all.append( [5, 6, 9, 6, 9, 6, 9, 6, 9, 6, 9, 8, 5] )
greatest_roads_all.append( [(6, 1, {'weight': 11}), (9, 5, {'weight': 81})] )
non_great_roads_all.append( [(0, 4, {'weight': 26}), (0, 6, {'weight': 78}), (0, 7, {'weight': 39}), (0, 9, {'weight': 91}), (1, 0, {'weight': 14}), (1, 9, {'weight': 23}), (2, 1, {'weight': 52}), (3, 1, {'weight': 55}), (3, 7, {'weight': 77}), (4, 8, {'weight': 13}), (5, 6, {'weight': 97}), (6, 0, {'weight': 4}), (6, 4, {'weight': 47}), (7, 3, {'weight': 3}), (7, 4, {'weight': 15}), (7, 8, {'weight': 87}), (7, 9, {'weight': 30}), (8, 5, {'weight': 10}), (9, 2, {'weight': 1}), (9, 3, {'weight': 18}), (9, 4, {'weight': 48}), (9, 7, {'weight': 45}), (9, 8, {'weight': 21})] )
path_all.append( [5, 6, 1, 0, 6, 1, 0, 6, 1, 0, 6, 1, 9, 5] )
greatest_roads_all.append( [(9, 5, {'weight': 92}), (9, 6, {'weight': 86})] )
non_great_roads_all.append( [(0, 7, {'weight': 95}), (1, 7, {'weight': 23}), (2, 4, {'weight': 80}), (3, 4, {'weight': 6}), (4, 2, {'weight': 68}), (4, 5, {'weight': 66}), (4, 6, {'weight': 4}), (4, 7, {'weight': 70}), (5, 1, {'weight': 90}), (5, 6, {'weight': 81}), (5, 8, {'weight': 73}), (5, 9, {'weight': 72}), (6, 8, {'weight': 86}), (7, 5, {'weight': 70}), (8, 3, {'weight': 56}), (8, 9, {'weight': 22}), (9, 0, {'weight': 46}), (9, 2, {'weight': 62})] )
path_all.append( [5, 9, 5, 9, 5, 9, 5, 9, 5, 9, 5] )
greatest_roads_all.append( [(4, 3, {'weight': 70}), (0, 3, {'weight': 91})] )
non_great_roads_all.append( [(0, 1, {'weight': 51}), (0, 2, {'weight': 56}), (1, 2, {'weight': 75}), (2, 0, {'weight': 95}), (2, 3, {'weight': 58}), (2, 6, {'weight': 79}), (2, 7, {'weight': 33}), (3, 4, {'weight': 37}), (4, 0, {'weight': 50}), (4, 9, {'weight': 45}), (5, 6, {'weight': 7}), (6, 0, {'weight': 36}), (6, 3, {'weight': 75}), (6, 7, {'weight': 45}), (7, 2, {'weight': 5}), (7, 6, {'weight': 87}), (7, 9, {'weight': 77}), (8, 4, {'weight': 98}), (8, 9, {'weight': 54}), (9, 3, {'weight': 22}), (9, 5, {'weight': 71}), (9, 6, {'weight': 80}), (9, 8, {'weight': 54})] )
path_all.append( [5, 6, 0, 3, 4, 3, 4, 3, 4, 3, 4, 3, 4, 9, 5] )
greatest_roads_all.append( [(0, 2, {'weight': 57}), (3, 7, {'weight': 67})] )
non_great_roads_all.append( [(0, 3, {'weight': 58}), (0, 4, {'weight': 3}), (0, 8, {'weight': 62}), (0, 9, {'weight': 39}), (1, 6, {'weight': 8}), (2, 1, {'weight': 20}), (2, 6, {'weight': 9}), (3, 2, {'weight': 32}), (3, 5, {'weight': 16}), (3, 9, {'weight': 54}), (4, 6, {'weight': 51}), (5, 2, {'weight': 8}), (6, 0, {'weight': 20}), (6, 2, {'weight': 70}), (6, 3, {'weight': 56}), (6, 4, {'weight': 22}), (7, 1, {'weight': 13}), (8, 1, {'weight': 96}), (8, 5, {'weight': 43}), (9, 7, {'weight': 46})] )
path_all.append( [5, 2, 6, 0, 2, 6, 0, 2, 6, 0, 2, 6, 0, 2, 6, 0, 2, 6, 3, 5] )
greatest_roads_all.append( [(9, 3, {'weight': 84})] )
non_great_roads_all.append( [(0, 2, {'weight': 60}), (0, 4, {'weight': 72}), (1, 4, {'weight': 55}), (2, 9, {'weight': 86}), (3, 0, {'weight': 2}), (4, 6, {'weight': 17}), (4, 7, {'weight': 64}), (4, 8, {'weight': 19}), (5, 9, {'weight': 59}), (6, 3, {'weight': 5}), (6, 8, {'weight': 28}), (7, 1, {'weight': 68}), (7, 4, {'weight': 100}), (7, 5, {'weight': 28}), (7, 6, {'weight': 64}), (8, 7, {'weight': 65}), (9, 4, {'weight': 25})] )
path_all.append( [5, 9, 3, 0, 2, 9, 3, 0, 2, 9, 3, 0, 2, 9, 3, 0, 2, 9, 3, 0, 4, 7, 5] )
greatest_roads_all.append( [(3, 2, {'weight': 38}), (0, 7, {'weight': 25})] )
non_great_roads_all.append( [(0, 1, {'weight': 18}), (0, 3, {'weight': 92}), (0, 9, {'weight': 36}), (1, 3, {'weight': 57}), (1, 5, {'weight': 15}), (2, 1, {'weight': 61}), (3, 1, {'weight': 67}), (4, 3, {'weight': 54}), (4, 8, {'weight': 4}), (5, 0, {'weight': 23}), (5, 1, {'weight': 56}), (5, 2, {'weight': 45}), (5, 7, {'weight': 5}), (5, 9, {'weight': 73}), (6, 0, {'weight': 75}), (6, 8, {'weight': 14}), (7, 4, {'weight': 4}), (7, 6, {'weight': 53}), (8, 9, {'weight': 74}), (9, 0, {'weight': 91}), (9, 2, {'weight': 25})] )
path_all.append( [5, 0, 7, 4, 3, 2, 1, 5, 0, 7, 6, 0, 7, 4, 3, 2, 1, 5] )
greatest_roads_all.append( [(2, 5, {'weight': 78}), (5, 0, {'weight': 71})] )
non_great_roads_all.append( [(0, 3, {'weight': 74}), (1, 2, {'weight': 6}), (1, 6, {'weight': 44}), (1, 8, {'weight': 3}), (2, 1, {'weight': 34}), (2, 6, {'weight': 10}), (3, 6, {'weight': 31}), (3, 9, {'weight': 41}), (4, 7, {'weight': 89}), (5, 1, {'weight': 12}), (5, 7, {'weight': 55}), (6, 7, {'weight': 27}), (7, 3, {'weight': 76}), (7, 4, {'weight': 65}), (7, 6, {'weight': 79}), (8, 2, {'weight': 1}), (8, 6, {'weight': 76}), (8, 7, {'weight': 75}), (8, 9, {'weight': 34}), (9, 0, {'weight': 76}), (9, 8, {'weight': 73})] )
path_all.append( [5, 1, 8, 2, 5, 1, 8, 2, 5, 1, 8, 2, 5, 1, 8, 2, 5, 1, 8, 2, 5] )
greatest_roads_all.append( [(9, 7, {'weight': 97}), (0, 8, {'weight': 5})] )
non_great_roads_all.append( [(1, 2, {'weight': 16}), (1, 3, {'weight': 48}), (1, 5, {'weight': 50}), (2, 0, {'weight': 51}), (2, 4, {'weight': 32}), (3, 1, {'weight': 80}), (3, 6, {'weight': 36}), (3, 8, {'weight': 55}), (4, 1, {'weight': 41}), (4, 5, {'weight': 68}), (5, 0, {'weight': 70}), (5, 2, {'weight': 54}), (5, 4, {'weight': 9}), (6, 1, {'weight': 22}), (6, 3, {'weight': 25}), (6, 7, {'weight': 15}), (7, 1, {'weight': 25}), (7, 3, {'weight': 61}), (7, 5, {'weight': 72}), (8, 1, {'weight': 56}), (8, 3, {'weight': 55}), (8, 9, {'weight': 23})] )
path_all.append( [5, 0, 8, 1, 2, 0, 8, 9, 7, 1, 2, 0, 8, 9, 7, 5] )
greatest_roads_all.append( [(6, 0, {'weight': 90})] )
non_great_roads_all.append( [(0, 1, {'weight': 38}), (1, 3, {'weight': 49}), (2, 6, {'weight': 27}), (3, 8, {'weight': 100}), (4, 2, {'weight': 20}), (4, 7, {'weight': 27}), (5, 1, {'weight': 52}), (6, 2, {'weight': 44}), (6, 7, {'weight': 22}), (7, 3, {'weight': 100}), (7, 5, {'weight': 13}), (7, 6, {'weight': 2}), (8, 9, {'weight': 36}), (9, 4, {'weight': 64}), (9, 6, {'weight': 15})] )
path_all.append( [5, 1, 3, 8, 9, 6, 0, 1, 3, 8, 9, 6, 0, 1, 3, 8, 9, 6, 0, 1, 3, 8, 9, 6, 0, 1, 3, 8, 9, 6, 0, 1, 3, 8, 9, 6, 7, 5] )
greatest_roads_all.append( [(2, 7, {'weight': 37}), (4, 1, {'weight': 16})] )
non_great_roads_all.append( [(0, 1, {'weight': 13}), (0, 9, {'weight': 62}), (1, 2, {'weight': 90}), (1, 6, {'weight': 4}), (1, 8, {'weight': 81}), (3, 1, {'weight': 33}), (3, 2, {'weight': 14}), (3, 7, {'weight': 21}), (3, 8, {'weight': 17}), (4, 0, {'weight': 64}), (4, 2, {'weight': 7}), (4, 6, {'weight': 12}), (5, 1, {'weight': 84}), (5, 2, {'weight': 1}), (5, 7, {'weight': 57}), (6, 0, {'weight': 79}), (6, 3, {'weight': 37}), (6, 5, {'weight': 85}), (6, 8, {'weight': 19}), (7, 1, {'weight': 1}), (7, 3, {'weight': 26}), (7, 4, {'weight': 34}), (8, 1, {'weight': 50}), (8, 5, {'weight': 36}), (9, 2, {'weight': 75})] )
path_all.append( [5, 2, 7, 4, 1, 6, 3, 2, 7, 3, 2, 7, 4, 1, 6, 8, 5] )

In [101]:
k = 5
n = 10
a = n//2

def is_great(u,v, greatest_roads):
    for a,b,w in greatest_roads:
        if a == u and b == v:
            return True
        if b == u and a == v:
            return True
    return False

for non_great_roads, greatest_roads, sol in zip(non_great_roads_all, greatest_roads_all, path_all):
    non_great_roads = [(edge[0],edge[1],edge[2]['weight']) for edge in non_great_roads]
    greatest_roads = [(edge[0],edge[1],edge[2]['weight']) for edge in greatest_roads]
    
    adj_list = [[-1 for j in range((k+1)*n)] for i in range((k+1)*n)]
    for u,v,w in non_great_roads:
        adj_list[u][v] = w
    for u,v,w in greatest_roads:
        adj_list[u][v] = w
        
    path = greatest_roads_solver(non_great_roads, greatest_roads, k, a, n)
    
    num_greatest_roads = 0
    for i in range(len(path)-1):
        assert adj_list[u][v] != -1
        if is_great(path[i], path[i + 1], greatest_roads):
            num_greatest_roads += 1
    
    assert num_greatest_roads >= k
    
    assert path[0] == sol[0]
    assert path[len(path) - 1] == sol[len(sol) - 1]
        
    distance1 = np.sum([adj_list[path[i]][path[i+1]] for i in range(len(path) - 1)])
    distance2 = np.sum([adj_list[sol[i]][sol[i+1]] for i in range(len(sol) - 1)])
    assert distance1 == distance2

print(ok)


         _          _                  _           _        
        /\ \       / /\               / /\        / /\      
       /  \ \     / /  \             / /  \      / /  \     
      / /\ \ \   / / /\ \           / / /\ \__  / / /\ \__  
     / / /\ \_\ / / /\ \ \         / / /\ \___\/ / /\ \___\ 
    / / /_/ / // / /  \ \ \        \ \ \ \/___/\ \ \ \/___/ 
   / / /__\/ // / /___/ /\ \        \ \ \       \ \ \       
  / / /_____// / /_____/ /\ \   _    \ \ \  _    \ \ \      
 / / /      / /_________/\ \ \ /_/\__/ / / /_/\__/ / /      
/ / /      / / /_       __\ \_\ \/___/ /  \ \/___/ /       
\/_/       \_\___\     /____/_/ \_____\/    \_____\/        
                                                            

